# Costruire un modello di regressione: preparare e visualizzare i dati

## **Regressione Lineare per le Zucche - Lezione 2**
#### Introduzione

Ora che hai a disposizione gli strumenti necessari per iniziare a costruire modelli di machine learning con Tidymodels e il Tidyverse, sei pronto per iniziare a porre domande ai tuoi dati. Quando lavori con i dati e applichi soluzioni di ML, è molto importante sapere come formulare la domanda giusta per sbloccare correttamente il potenziale del tuo dataset.

In questa lezione, imparerai:

- Come preparare i tuoi dati per la costruzione di modelli.

- Come utilizzare `ggplot2` per la visualizzazione dei dati.

La domanda a cui hai bisogno di rispondere determinerà il tipo di algoritmi di ML che utilizzerai. E la qualità della risposta che otterrai dipenderà fortemente dalla natura dei tuoi dati.

Vediamo questo concetto attraverso un esercizio pratico.

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Illustrazione di @allison_horst</figcaption>


<!--![Illustrazione di \@allison_horst](../../../../../../translated_images/unruly_data.0eedc7ced92d2d919cf5ea197bfe0fe9a30780c4bf7cdcf14ff4e9dc5a4c7267.it.jpg)<br>Illustrazione di \@allison_horst-->


## 1. Importare i dati delle zucche e richiamare il Tidyverse

Avremo bisogno dei seguenti pacchetti per analizzare e manipolare questa lezione:

-   `tidyverse`: Il [tidyverse](https://www.tidyverse.org/) è una [collezione di pacchetti R](https://www.tidyverse.org/packages) progettata per rendere la scienza dei dati più veloce, semplice e divertente!

Puoi installarli con il comando:

`install.packages(c("tidyverse"))`

Lo script qui sotto verifica se hai i pacchetti necessari per completare questo modulo e li installa per te nel caso in cui ne manchi qualcuno.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Ora, avviamo alcuni pacchetti e carichiamo i [dati](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) forniti per questa lezione!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Un rapido `glimpse()` mostra immediatamente che ci sono valori mancanti e un mix di stringhe (`chr`) e dati numerici (`dbl`). La colonna `Date` è di tipo carattere e c'è anche una colonna strana chiamata `Package`, dove i dati sono un mix tra `sacks`, `bins` e altri valori. I dati, in effetti, sono un po' un disastro 😤.

In realtà, non è molto comune ricevere un dataset completamente pronto per essere utilizzato per creare un modello di ML direttamente. Ma non preoccuparti, in questa lezione imparerai come preparare un dataset grezzo utilizzando le librerie standard di R 🧑‍🔧. Imparerai anche diverse tecniche per visualizzare i dati. 📈📊
<br>

> Un ripasso: L'operatore pipe (`%>%`) esegue operazioni in sequenza logica passando un oggetto avanti in una funzione o espressione. Puoi pensare all'operatore pipe come se dicesse "e poi" nel tuo codice.


## 2. Controllare i dati mancanti

Uno dei problemi più comuni che i data scientist devono affrontare è la presenza di dati incompleti o mancanti. R rappresenta i valori mancanti, o sconosciuti, con un valore sentinella speciale: `NA` (Not Available).

Quindi, come possiamo sapere se il data frame contiene valori mancanti?  
<br>
-   Un modo semplice sarebbe utilizzare la funzione base di R `anyNA`, che restituisce gli oggetti logici `TRUE` o `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Ottimo, sembra che manchino alcuni dati! Questo è un buon punto di partenza.

-   Un altro modo sarebbe utilizzare la funzione `is.na()` che indica quali elementi individuali delle colonne sono mancanti con un valore logico `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Ok, lavoro fatto, ma con un data frame così grande, sarebbe inefficiente e praticamente impossibile esaminare tutte le righe e le colonne singolarmente😴.

-   Un modo più intuitivo sarebbe calcolare la somma dei valori mancanti per ogni colonna:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Molto meglio! Mancano alcuni dati, ma forse non sarà un problema per il compito da svolgere. Vediamo cosa porterà avanti un'ulteriore analisi.

> Oltre ai fantastici set di pacchetti e funzioni, R ha una documentazione molto valida. Ad esempio, usa `help(colSums)` o `?colSums` per scoprire di più sulla funzione.


## 3. Dplyr: Una grammatica per la manipolazione dei dati

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Illustrazione di @allison_horst</figcaption>


<!--![Illustrazione di \@allison_horst](../../../../../../translated_images/dplyr_wrangling.f5f99c64fd4580f1377fee3ea428b6f8fd073845ec0f8409d483cfe148f0984e.it.png)<br/>Illustrazione di \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), un pacchetto del Tidyverse, è una grammatica per la manipolazione dei dati che offre un insieme coerente di verbi per aiutarti a risolvere le sfide più comuni nella manipolazione dei dati. In questa sezione, esploreremo alcuni dei verbi di dplyr!  
<br>


#### dplyr::select()

`select()` è una funzione del pacchetto `dplyr` che ti aiuta a scegliere le colonne da mantenere o escludere.

Per rendere il tuo data frame più facile da gestire, elimina alcune delle sue colonne utilizzando `select()`, mantenendo solo le colonne di cui hai bisogno.

Ad esempio, in questo esercizio, la nostra analisi riguarderà le colonne `Package`, `Low Price`, `High Price` e `Date`. Selezioniamo queste colonne.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` è una funzione del pacchetto `dplyr` che ti aiuta a creare o modificare colonne, mantenendo intatte le colonne esistenti.

La struttura generale di `mutate` è:

`data %>%   mutate(new_column_name = what_it_contains)`

Proviamo a utilizzare `mutate` con la colonna `Date` eseguendo le seguenti operazioni:

1. Convertire le date (attualmente di tipo carattere) in un formato mese (si tratta di date statunitensi, quindi il formato è `MM/DD/YYYY`).

2. Estrarre il mese dalle date in una nuova colonna.

In R, il pacchetto [lubridate](https://lubridate.tidyverse.org/) semplifica il lavoro con i dati di tipo Date-time. Quindi, utilizziamo `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` e vediamo come raggiungere gli obiettivi sopra indicati. Possiamo eliminare la colonna `Date` poiché non ci servirà più nelle operazioni successive.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Evviva! 🤩

Ora, creiamo una nuova colonna `Price`, che rappresenta il prezzo medio di una zucca. Adesso, calcoliamo la media delle colonne `Low Price` e `High Price` per riempire la nuova colonna Price.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Sììì! 💪

"Ma aspetta!", dirai dopo aver dato un'occhiata all'intero set di dati con `View(pumpkins)`, "C'è qualcosa di strano qui!" 🤔

Se guardi la colonna `Package`, le zucche vengono vendute in molte configurazioni diverse. Alcune sono vendute in misure di `1 1/9 bushel`, altre in misure di `1/2 bushel`, alcune per zucca, altre per libbra, e altre ancora in grandi scatole di larghezze variabili.

Verifichiamolo:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Fantastico!👏

Le zucche sembrano essere molto difficili da pesare in modo coerente, quindi filtriamole selezionando solo le zucche con la stringa *bushel* nella colonna `Package` e inseriamole in un nuovo data frame `new_pumpkins`.


#### dplyr::filter() e stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): crea un sottoinsieme dei dati contenente solo le **righe** che soddisfano le tue condizioni, in questo caso, zucche con la stringa *bushel* nella colonna `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): rileva la presenza o l'assenza di un pattern in una stringa.

Il pacchetto [`stringr`](https://github.com/tidyverse/stringr) fornisce funzioni semplici per operazioni comuni sulle stringhe.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Puoi vedere che abbiamo ristretto a circa 415 righe di dati contenenti zucche al moggio.🤩  
<br>


#### dplyr::case_when()

**Ma aspetta! C'è ancora una cosa da fare**

Hai notato che la quantità di bushel varia per riga? Devi normalizzare i prezzi in modo da mostrare il prezzo per bushel, non per 1 1/9 o 1/2 bushel. È il momento di fare un po' di calcoli per standardizzarlo.

Utilizzeremo la funzione [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) per *modificare* la colonna Price in base a determinate condizioni. `case_when` ti permette di vettorializzare più istruzioni `if_else()`.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Ora possiamo analizzare il prezzo per unità basandoci sulla loro misura in staia. Tutto questo studio sulle staia di zucche, tuttavia, dimostra quanto sia `importante` `comprendere la natura dei propri dati`!

> ✅ Secondo [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), il peso di una staia dipende dal tipo di prodotto, poiché si tratta di una misura di volume. "Una staia di pomodori, ad esempio, dovrebbe pesare 56 libbre... Le foglie e le verdure occupano più spazio con meno peso, quindi una staia di spinaci pesa solo 20 libbre." È tutto piuttosto complicato! Non preoccupiamoci di fare una conversione da staia a libbre, e invece calcoliamo il prezzo per staia. Tutto questo studio sulle staia di zucche, tuttavia, dimostra quanto sia importante comprendere la natura dei propri dati!
>
> ✅ Hai notato che le zucche vendute a mezza staia sono molto costose? Riesci a capire il perché? Suggerimento: le zucche piccole sono molto più care di quelle grandi, probabilmente perché ce ne sono molte di più per staia, dato lo spazio inutilizzato occupato da una grande zucca vuota per torte.


Ora, per puro spirito d'avventura 💁‍♀️, spostiamo anche la colonna Mese nella prima posizione, cioè `prima` della colonna `Pacchetto`.

Si utilizza `dplyr::relocate()` per modificare la posizione delle colonne.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Ottimo lavoro!👌 Ora hai un dataset pulito e ordinato su cui puoi costruire il tuo nuovo modello di regressione!
<br>


## 4. Visualizzazione dei dati con ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infografica di Dasani Madipalli</figcaption>


<!--![Infografica di Dasani Madipalli](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.it.png){width="600"}-->

C'è un *saggio* detto che recita così:

> "Il semplice grafico ha portato più informazioni alla mente dell'analista di dati di qualsiasi altro strumento." --- John Tukey

Parte del ruolo del data scientist è dimostrare la qualità e la natura dei dati con cui sta lavorando. Per farlo, spesso crea visualizzazioni interessanti, come grafici, diagrammi e chart, che mostrano diversi aspetti dei dati. In questo modo, è in grado di mostrare visivamente relazioni e lacune che altrimenti sarebbero difficili da individuare.

Le visualizzazioni possono anche aiutare a determinare la tecnica di machine learning più appropriata per i dati. Un diagramma a dispersione che sembra seguire una linea, ad esempio, indica che i dati sono un buon candidato per un esercizio di regressione lineare.

R offre diversi sistemi per creare grafici, ma [`ggplot2`](https://ggplot2.tidyverse.org/index.html) è uno dei più eleganti e versatili. `ggplot2` consente di comporre grafici **combinando componenti indipendenti**.

Iniziamo con un semplice diagramma a dispersione per le colonne Price e Month.

In questo caso, inizieremo con [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), forniremo un dataset e una mappatura estetica (con [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) e poi aggiungeremo un livello (come [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) per i diagrammi a dispersione.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

È un grafico utile 🤷? C'è qualcosa che ti sorprende?

Non è particolarmente utile, dato che tutto ciò che fa è mostrare i tuoi dati come una distribuzione di punti in un determinato mese.
<br>


### **Come lo rendiamo utile?**

Per visualizzare dati utili nei grafici, di solito è necessario raggruppare i dati in qualche modo. Ad esempio, nel nostro caso, calcolare il prezzo medio delle zucche per ogni mese fornirebbe maggiori informazioni sui modelli sottostanti nei nostri dati. Questo ci porta a un'altra panoramica di **dplyr**:

#### `dplyr::group_by() %>% summarize()`

L'aggregazione raggruppata in R può essere facilmente calcolata utilizzando

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` cambia l'unità di analisi dal dataset completo ai singoli gruppi, come per mese.

-   `dplyr::summarize()` crea un nuovo data frame con una colonna per ogni variabile di raggruppamento e una colonna per ciascuna delle statistiche di riepilogo che hai specificato.

Ad esempio, possiamo utilizzare `dplyr::group_by() %>% summarize()` per raggruppare le zucche in base alla colonna **Month** e poi calcolare il **prezzo medio** per ogni mese.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Succinto!✨

Le caratteristiche categoriche, come i mesi, sono meglio rappresentate utilizzando un grafico a barre 📊. I livelli responsabili per i grafici a barre sono `geom_bar()` e `geom_col()`. Consulta `?geom_bar` per saperne di più.

Prepariamone uno!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩 Questa è una visualizzazione dei dati più utile! Sembra indicare che il prezzo più alto delle zucche si verifica a settembre e ottobre. Rispecchia le tue aspettative? Perché sì o perché no?

Congratulazioni per aver completato la seconda lezione 👏! Hai preparato i tuoi dati per la costruzione del modello, poi hai scoperto ulteriori approfondimenti utilizzando le visualizzazioni!



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
